# Create tables with p-value and change in medians for multiple cancers 

Create a dataframe with p-value results from t-tests for all proteins (trans gene proteomics when PTEN has cnv deletions compared to PTEN wildtype). The dataframe also includes the change in medians between deletions and wildtype. Prepare these tables for further analysis by creating csv files.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats
import gseapy as gp
import re
import sys 

import cptac
import cptac.utils as u

import plot_utils as p
root = R'~\Github\WhenMutationsDontMatter\PTEN\Figure_2\csv'

In [2]:
#import warnings
#warnings.filterwarnings('ignore')

In [57]:
import pandas as pd
import numpy as np
import scipy.stats
import statsmodels.stats.multitest
import operator


def wrap_ttest(df, label_column, comparison_columns=None, alpha=.05, return_all=False, 
               correction_method='bonferroni', mincount=3, pval_return_corrected=True):
    try:
        '''Verify precondition that label column exists and has exactly 2 unique values'''
        label_values = df[label_column].unique()
        if len(label_values) != 2:
            print("Incorrectly Formatted Dataframe! Label column must have exactly 2 unique values.")
            return None
       
        '''Partition dataframe into two sets, one for each of the two unique values from the label column'''
        partition1 = df.loc[df[label_column] == label_values[0]]
        partition2 = df.loc[df[label_column] == label_values[1]]

        '''If no comparison columns specified, use all columns except the specified labed column'''
        if not comparison_columns:
            comparison_columns = list(df.columns)
            comparison_columns.remove(label_column)

        '''Determine the number of real valued columns on which we will do t-tests'''
        number_of_comparisons = len(comparison_columns)

        '''Store comparisons and p-values in two arrays'''
        comparisons = []
        pvals = []
       
        '''Loop through each comparison column, perform the t-test, and record the p-val'''
       
        for column in comparison_columns:
            if len(partition1[column].dropna(axis=0)) <= mincount:
                continue
            elif len(partition2[column].dropna(axis=0)) <= mincount:
                continue
            else:
                stat, pval = scipy.stats.ttest_ind(partition1[column].dropna(axis=0), partition2[column].dropna(axis=0))
                comparisons.append(column)
                pvals.append(pval)
               
        '''Correct for multiple testing to determine if each comparison meets the new cutoff'''
        results = statsmodels.stats.multitest.multipletests(pvals=pvals, alpha=alpha, method=correction_method)
        reject = results[0]

        '''Format results in a pandas dataframe'''
        results_df = pd.DataFrame(columns=['Comparison','P_Value'])

        '''If return all, add all comparisons and p-values to dataframe'''
        if return_all:
            if pval_return_corrected:
                results_df['Comparison'] = comparisons
                results_df['P_Value'] = results[1]

            else:
                results_df['Comparison'] = comparisons
                results_df['P_Value'] = pvals

            '''Else only add significant comparisons'''
        else:
            for i in range(0, len(reject)):
                if reject[i]:
                    if pval_return_corrected:
                        results_df = results_df.append({'Comparison':comparisons[i],'P_Value':results[1][i]}, ignore_index=True)
                    else:
                        results_df = results_df.append({'Comparison':comparisons[i],'P_Value':pvals[i]}, ignore_index=True)


        '''Sort dataframe by ascending p-value'''
        results_df = results_df.sort_values(by='P_Value', ascending=True)
        results_df = results_df.reset_index(drop=True)

        '''If results df is not empty, return it, else return None'''
        if len(results_df) > 0:
            return results_df
        else:
            return None


    except:
        print("Incorrectly Formatted Dataframe!")
        return None



In [4]:
# Returns a dataframe with proteomics and mutation type

# all_prot: list of trans genes

def all_prot_format_df(cancer_object, mir_list, gene_in = 'PTEN', utils = u):
    mut_type = cancer_object.get_genotype_all_vars(gene_in)
    
    if cancer_object.get_cancer_type() not in ('luad'):
        # Keep only tumor samples from proteomics
        prot_and_mutations = cancer_object.join_omics_to_mutations(
            mutations_genes = [gene_in], omics_df_name = 'proteomics', omics_genes = [gene_in])
        # Reduce a multiindex 
        if isinstance(prot_and_mutations.keys(), pd.core.indexes.multi.MultiIndex):
            prot_and_mutations = utils.reduce_multiindex(prot_and_mutations, levels_to_drop = 1)
        prot_and_mutations = prot_and_mutations[prot_and_mutations.Sample_Status == "Tumor"] # drop Normal samples

        # Merge Mutation column from get_genotype_all_vars (includes cnv) with proteomics
        mut_type = mut_type[['Mutation']] 
        prot_df = prot_and_mutations.iloc[:,:-4] # Keep only proteomics
        merged = prot_df.join(mut_type)
        
        # Keep only Wildtype and deletion
        compare = ['Wildtype_Tumor','Deletion']
        get = merged['Mutation'].isin(compare)
        del_wt = merged[get]
    
    
    # Luad has no somatic mutations for PTEN which changes some things
    else: 
        # get_genotype_all_vars add cnv data under the column PTEN
        mut_type = mut_type.drop(columns= gene_in)
        # different code because no somatic mutation data for pten (can't join to somatic mutations)
        omics = cancer_object.join_omics_to_omics(df1_name = 'CNV', df2_name='proteomics',genes1= gene_in, 
            genes2= gene_in)
        omics = utils.reduce_multiindex(omics, levels_to_drop = 1, flatten = True)
        omics = omics.drop(columns='PTEN_CNV')
        # Get only tumor samples
        p = cancer_object.get_proteomics(tissue_type='tumor')
        tumor_ids = list(p.index)
        get = omics.index.isin(tumor_ids)
        omics = omics[get]
        # Merge Mutation column from get_genotype_all_vars (includes cnv) with proteomics
        merged = omics.join(mut_type) 
        # Keep only No_Mutation (wildtype) and deletion
        compare = ['No_Mutation','Deletion']
        get = merged['Mutation'].isin(compare)
        del_wt = merged[get]
        del_wt['Mutation'] = np.where(
            del_wt['Mutation'] == 'No_Mutation', 'Wildtype_Tumor', 'Deletion')

    return del_wt


In [5]:
'''
Params
cancer_object: Object. Variable for the loaded cancer data set.
cancer_name: Str. name to add to the created dataframe.
del_wt_df: DataFrame. Only samples with deletions and wildtype for PTEN. 
all_prot_list: List. All proteins in proteomics data frame. 

Returns a dataframe with the difference in medians between proteomics with PTEN wt and del (wt - del). 
'''

def get_change_in_medians_df(cancer_object, cancer_name, del_wt_df, all_prot_list):
    d = del_wt_df[del_wt_df.Mutation == "Deletion"]
    wt = del_wt_df[del_wt_df.Mutation == "Wildtype_Tumor"]
    del_med = d.median()
    wt_med = wt.median()

    med_dict = {}
    
    # Correlation: + is mutant up compared to wt, - is mutant down
    for prot in all_prot_list:
        if prot in del_med.index and prot in wt_med.index:
            dif = del_med[prot] - wt_med[prot]
            med_dict[prot] = dif
        
    df = pd.DataFrame.from_dict(med_dict, orient='index', columns=[cancer_name+'_Median'])
    df = df.reset_index().rename(columns={'index':'Proteomics'})
    
    return df
        

#  Step 1: Create Data frames with p-values and differences in median

Each cancer needs a data frame containing only samples that have PTEN cnv deletions and PTEN wildtype with trans proteomics. Use wrap_ttest to run many T-tests for all genes in the proteomics data frame. Use get_change_in_medians_df to create the data frame with change in median values. Merge both data frames. 

Load in cancer data sets from cptac. 

In [6]:
g = cptac.Gbm()
en = cptac.Endometrial()
h = cptac.Hnscc()
l = cptac.Luad()
ls = cptac.Lscc()
o = cptac.Ovarian()
col = cptac.Colon()
b = cptac.Brca()

cptac warning: The GBM dataset is under publication embargo until March 01, 2021. CPTAC is a community resource project and data are made available rapidly after generation for community research use. The embargo allows exploring and utilizing the data, but analysis may not be published until after the embargo date. Please see https://proteomics.cancer.gov/data-portal/about/data-use-agreement or enter cptac.embargo() to open the webpage for more details. (C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\ipykernel_launcher.py, line 1)


cptac warning: The HNSCC data is currently strictly reserved for CPTAC investigators. Otherwise, you are not authorized to access these data. Additionally, even after these data become publicly available, they will be subject to a publication embargo (see https://proteomics.cancer.gov/data-portal/about/data-use-agreement or enter cptac.embargo() to open the webpage for more details). (C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\ipykernel_launcher.py, line 3)


version 3scc v3.2.......                 


cptac warning: The LSCC data is currently strictly reserved for CPTAC investigators. Otherwise, you are not authorized to access these data. Additionally, even after these data become publicly available, they will be subject to a publication embargo (see https://proteomics.cancer.gov/data-portal/about/data-use-agreement or enter cptac.embargo() to open the webpage for more details). (C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\ipykernel_launcher.py, line 5)


# Find the average of miRNA abundance

In [15]:
gm = g.get_miRNA()
gm = u.reduce_multiindex(gm, levels_to_drop = 1)
m25 = gm[['hsa-miR-25-3p']]

median = float(m25.median())
q1 = np.percentile(m25, 25, interpolation = 'nearest') 
q3 = np.percentile(m25, 75, interpolation = 'nearest') 

print('q1:', q1, '\tmedian:', median, '\tq3:', q3)

q1: 229.9597570425176 	median: 317.4203759908164 	q3: 432.57632476499464


cptac warning: Due to dropping the specified levels, dataframe now has 400 duplicated column headers. (C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\ipykernel_launcher.py, line 2)


In [8]:
sd = float(m25.std()) # (median + sd) - (med - sd) = 68.2 ???

In [16]:
gene = 'PTEN'
gm_list = list(m25.columns)
gm = all_prot_format_df(g, gm_list)
gm.Mutation.value_counts()
gm

cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 72 samples for the PTEN gene (C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\cptac\dataset.py, line 313)
cptac warning: CNV data was not found for the following samples, so CNV data columns were filled with NaN for these samples: C3L-01834 (C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\cptac\dataset.py, line 313)
cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 82 samples for the PTEN gene (C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\ipykernel_launcher.py, line 11)


Name,PTEN_proteomics,Mutation
Patient_ID,,
C3L-00104,-0.431991,Deletion
C3L-00365,-0.265044,Deletion
C3L-00674,0.390190,Deletion
C3L-00677,-0.915526,Deletion
C3L-01040,0.680291,Deletion
...,...,...
C3N-03183,-0.725120,Deletion
C3N-03184,-0.242603,Deletion
C3N-03186,0.428594,Deletion


In [17]:
def status(row):
    status = 'na'
    for val in row: 
        if pd.isnull(val):
            continue
        val = float(val)
        if val < q1:
            status = 'low'
        elif val < q3:
            status = 'high'
        else:
            status = 'med'
        return status


m25['miRNA_status'] = m25.apply(status, axis = 1)
m25

Name,hsa-miR-25-3p,miRNA_status
Patient_ID,,
C3L-00104,341.652573,high
C3L-00365,343.139007,high
C3L-00674,1428.981227,med
C3L-00677,890.645621,med
C3L-01040,164.881399,low
...,...,...
PT-RN5K.N,83.473228,low
PT-RU72.N,67.660219,low
PT-UTHO.N,150.314215,low


In [49]:
t = m25.index.isin(['C3N-01856'])
#m25[m25.index.str.contains(r'C3N-0185')]

Name,hsa-miR-25-3p,miRNA_status
Patient_ID,,
C3N-01851,310.030826,high
C3N-01852,158.909127,low
C3N-01857,361.351454,high


In [50]:
both = gm[['PTEN_proteomics']].join(m25[['miRNA_status']])
hilo = both.loc[both['miRNA_status'] != 'med'].dropna()

In [53]:
g_cols = ['PTEN_proteomics']

# Get all pvals
g_pval = wrap_ttest(hilo, 'miRNA_status', return_all = True, correction_method = 'fdr_bh', mincount = 3)
#g_pval = g_pval.rename(columns = {'Comparison': 'Proteomics','P_Value': 'Gbm_P_Value'})
g_cols
hilo['miRNA_status'].value_counts()

high    51
low     18
Name: miRNA_status, dtype: int64

In [55]:
g_pval

,Comparison,P_Value
0,PTEN_proteomics,0.118912


Part 2: Run T-tests. Create a data frame of just significant comparisons and another data frame for all comparisons. A moderately stringent correction is used (FDR_BH).

In [20]:
g_cols = ['PTEN_proteomics']

# Get all pvals
g_pval = wrap_ttest(hilo, 'miRNA_status', g_cols, return_all = True, correction_method = 'fdr_bh', mincount = 3)
#g_pval = g_pval.rename(columns = {'Comparison': 'Proteomics','P_Value': 'Gbm_P_Value'})
g_cols
hilo['miRNA_status'].value_counts()

t 
Incorrectly Formatted Dataframe! Label column must have exactly 2 unique values.


high    51
low     18
Name: miRNA_status, dtype: int64

In [13]:
hilo


Name,PTEN_proteomics,miRNA_status
Patient_ID,,
C3L-00104,-0.431991,high
C3L-00365,-0.265044,high
C3L-01040,0.680291,low
C3L-01043,-0.580510,high
C3L-01046,-0.341718,high
...,...,...
C3N-03088,0.955729,high
C3N-03180,0.273347,high
C3N-03184,-0.242603,high


Part 3: Create the differences in median df. (median of wildtype tumors - median of cnv deletions)

In [14]:
g_med = get_change_in_medians_df(g, "Gbm", g_del_wt, g_cols)

NameError: name 'g_del_wt' is not defined

Part 4: Merge the p-values and the differences in median dfs.

In [ ]:
g_merged = g_pval.merge(g_med, on='Proteomics', how='outer')
g_merged = g_merged.replace(to_replace = r'_proteomics', value = '', regex = True)

# Create csv
g_merged.to_csv(root+R'\Gbm_pval_medians.csv',index=False)
g_merged.head()

# Repeat for other cancers.

In [ ]:
cancer_objects = {'Hnscc':h, 'Luad':l, 'Lscc':ls, 'Brca':b, 'Colon':col, 'Ov':o}

In [ ]:
import warnings
warnings.filterwarnings('ignore')

gene = 'PTEN'
merged_dfs = {}
for cancer in cancer_objects:

    prot = cancer_objects[cancer].get_proteomics()
    prot = prot.drop(columns = 'PTEN')
    # Reduce a multiindex 
    if isinstance(prot.keys(), pd.core.indexes.multi.MultiIndex):
        prot = u.reduce_multiindex(prot, levels_to_drop = 1)
    prot_list = list(prot.columns)
    
    # Format df for t-test
    del_wt = all_prot_format_df(cancer_objects[cancer], prot_list)
    #del_wt = del_wt.dropna(axis='columns', thresh = 10)
    
    # Rename duplicate columns (isoforms)
    col_names = pd.Series(del_wt.columns[:])
    for dup in col_names[col_names.duplicated()].unique(): 
        col_names[col_names[col_names == dup].index.values.tolist()] = [dup + '_isoform_' + str(i) if i != 0 else dup for i in range(sum(col_names == dup))]
    del_wt.columns = col_names # rename the columns with the cols list
    
    # T-test
    cols_in = list(del_wt.columns[:-1])

    # Get all pvals
    min_num = 5
    all_pval = wrap_ttest(del_wt, 'Mutation', cols_in, return_all = True, correction_method = 'fdr_bh', mincount = min_num)
    all_pval = all_pval.rename(columns = {'Comparison': 'Proteomics','P_Value': cancer+'_P_Value'})
    
    # Get difference in medians
    delta_median_df = get_change_in_medians_df(cancer_objects[cancer], cancer, del_wt, cols_in)

    # Merge pval_df and delta_median_df
    pval_medians_df = all_pval.merge(delta_median_df, on='Proteomics', how='outer')
    pval_medians_df = pval_medians_df.replace(to_replace = r'_proteomics', value = '', regex = True)
    merged_dfs[cancer] = pval_medians_df # testing purposes
    
    # Create csv
    pval_medians_df.to_csv(root+'\\'+cancer+'_pval_medians.csv', index=False)
    print(pval_medians_df.head(), '\n')

# Endometrial

The Endometrial data set does not have enough cnv deletions to perform a t-test, however the data set does have enough truncation type mutations (nonsense and frameshift). Different code is needed to create the data frame for Endometrial.

In [ ]:
gene = 'PTEN'
prot = en.get_proteomics()
prot = prot.drop(columns = 'PTEN')
e_prot_list = list(prot.columns)

mut_type = en.get_genotype_all_vars(gene)

# merge cnv with genotype all mut type
prot_and_mutations = en.join_omics_to_mutations(
    mutations_genes = [gene], omics_df_name = 'proteomics', omics_genes = e_prot_list)
prot_and_mutations = prot_and_mutations[prot_and_mutations.Sample_Status == "Tumor"] # drop Normal samples
ip_df = prot_and_mutations.iloc[:,:-4] #drop mutation, location cols
mut_type = mut_type[['Mutation']] 
merged = ip_df.join(mut_type) # merge mutation col from function (includes cnv)

# Keep two values to compare
compare = ['Wildtype_Tumor','Nonsense_Mutation','Frame_Shift_Ins','Frame_Shift_Del']
get = merged['Mutation'].isin(compare)
trunc_wt = merged[get]
trunc_wt['Mutation'] = np.where(
                trunc_wt['Mutation'] == 'Wildtype_Tumor', 'Wildtype_Tumor', 'Truncation')

In [ ]:
e_cols = list(trunc_wt.columns[:-1])

# Get all pvals
e_pval = wrap_ttest(trunc_wt, 'Mutation', e_cols, return_all = True, correction_method = 'fdr_bh', mincount = 5)
e_pval = e_pval.rename(columns = {'Comparison': 'Proteomics','P_Value': 'En_P_Value'})

Differences in median with adaption for trunctation mutations.

In [ ]:
t = trunc_wt[trunc_wt.Mutation == "Truncation"]
wt = trunc_wt[trunc_wt.Mutation == "Wildtype_Tumor"]
trunc_med = t.median()
wt_med = wt.median()

en_d = {}

for prot in e_cols:
    if prot in trunc_med.index and prot in wt_med.index:
        dif = trunc_med[prot] - wt_med[prot]
        en_d[prot] = dif

en_med = pd.DataFrame.from_dict(en_d, orient='index', columns=['En_Median'])
en_med = en_med.reset_index().rename(columns={'index':'Proteomics'})

In [ ]:
e_merged = e_pval.merge(en_med, on='Proteomics',how='outer')
e_merged = e_merged.replace(to_replace = r'_proteomics', value = '', regex = True)

# Create csv
e_merged.to_csv(root+R'\En_pval_medians.csv',index=False)
e_merged.head()

# Get number of significant genes in each cancer

In [ ]:
# Add Gbm and En to merged_dfs dictionary
merged_dfs['Gbm'] = g_merged
merged_dfs['En'] = e_merged

In [ ]:
# Using corrected pvals from wrap_ttest, each cancer's cutoff is 0.05
all_sig = []

for cancer in merged_dfs:
    df = merged_dfs[cancer]
    sig_df = df.loc[df[cancer+'_P_Value'] < 0.05]
    print(cancer, 'sig comparisons:', len(sig_df))
    sig_list = list(sig_df['Proteomics'])
    all_sig.append(sig_list)
        

flat_list = [item for sublist in all_sig for item in sublist] #change list of lists, to just one list
sig = list(set(flat_list)) # remove duplicates
print('\nNumber of significant comparisons in at least 1 cancer:', len(sig))

In [ ]:
# check

In [ ]:
# check
lu = merged_dfs['Luad']
len(lu.Proteomics.unique())
sl = lu.loc[lu['Luad_P_Value'] < 0.05]
list(sl.Proteomics)
lu.loc[lu['Proteomics'] == 'PIK3R1_isoform_1'] #ITGAL_isoform_1, PIK3R1_isoform_1, RABGAP1L_isoform_1

In [ ]:
# check
pr = all_prot_format_df(l, all_prot = ['RABGAP1L'])

col_names = pd.Series(pr.columns)

for dup in col_names[col_names.duplicated()].unique(): 
    col_names[col_names[col_names == dup].index.values.tolist()] = [dup + '_isoform_' + str(i) if i != 0 else dup for i in range(sum(col_names == dup))]
pr.columns = col_names # rename the columns with the cols list

pr = pr[['RABGAP1L_proteomics_isoform_1', 'Mutation']]
delt = pr.loc[pr['Mutation'] == 'Deletion']
wt = pr.loc[pr['Mutation'] != 'Deletion']

dm = float(delt.median())
wtm = float(wt.median())


print(dm, '-', wtm, '=', dm - wtm)



In [ ]:
# check

# Get a list of significant genes in at least one cancer

In [ ]:
s = pd.Series(sig)
print('Number of significant genes in at least one cancer:', len(s))
s.to_csv(root+R'\list_sig_one_cancer.csv', index=False)

# Get a list of significant genes in multiple cancers

In [ ]:
# sig in multiple (more than 1) cancers
mult = list(set(i for i in flat_list if flat_list.count(i) > 1)) # Keep genes sig in more than 1 cancer
print('Number of significant genes in mult cancers:', len(mult))

In [ ]:
m = pd.Series(mult)
m = m.replace(to_replace = r'_proteomics$', value = '', regex = True)
m.to_csv(root+R'\list_sig_multiple_cancers.csv', index=False)